In [9]:
%matplotlib inline
import urllib
import os
import openpyxl as px
import zipfile
import csv
import pandas as pd
import numpy as np

from __future__ import division

In [10]:
urls = ["https://resources.lendingclub.com/LoanStats3a.csv.zip",
       "https://resources.lendingclub.com/LoanStats3b.csv.zip",
        "https://resources.lendingclub.com/LoanStats3c.csv.zip",
        "https://resources.lendingclub.com/LoanStats3d.csv.zip"]

data_dict_url = "https://resources.lendingclub.com/LCDataDictionary.xlsx"

def download(url, unzip=False):
    file_name = url.split("/")[-1]
    if os.path.isfile(file_name):
        print "%s already downloaded" % file_name
    else:    
        print "Downloading %s..." % file_name
        urllib.urlretrieve(url, file_name)
        print "%s downloaded" % file_name

    if unzip:
        file_type = file_name.split(".")[-1]
        if file_type.lower() == "zip":
            z = zipfile.ZipFile(file_name)
            extracted = []
            for f in z.namelist():
                extracted.append(f)
                if os.path.isfile(f):
                    print "%s already unzipped" % f
                else:
                    z.extract(f)
                    print "Unzipped %s" % f
            return extracted
    return file_name

headers_file = download(data_dict_url)
headers_ws = px.load_workbook(headers_file, read_only=True).get_active_sheet()

data_files = []

for url in urls:
    data_file = download(url, unzip=True)
    data_files.extend(data_file)

LCDataDictionary.xlsx already downloaded
LoanStats3a.csv.zip downloaded
Unzipped LoanStats3a.csv
LoanStats3b.csv.zip downloaded
Unzipped LoanStats3b.csv
LoanStats3c.csv.zip downloaded
Unzipped LoanStats3c.csv
LoanStats3d.csv.zip downloaded
Unzipped LoanStats3d.csv


In [3]:
# Deal with the headers

# Dispense of header of headers
headers_ws.rows.next()

headers = []
headers_desc = {}
for row in headers_ws.rows:
    header, desc = map(lambda x: x.value, row[:2])
    if header is not None:
        headers.append(header)
        headers_desc[header] = desc

In [4]:
# Load the data
df = pd.DataFrame()
for data_file in data_files:
    with open(data_file,'rb') as f:
        reader = csv.reader(f)
        # remove disclaimer
        reader.next()
        
        header_row = 1
        end_row = 1
        while len(reader.next()) > 1:
            end_row += 1
        print "Loading %s" % data_file
        df = df.append(pd.read_csv(data_file, header=header_row, nrows=end_row))
        print "Finished loading %s" % data_file

print "Finished loading data."

Loading LoanStats3a.csv
Finished loading LoanStats3a.csv
Finished loading data.


/Users/Branko/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2705: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def mdiff(start, end):
    """ Difference between two dates in months """
    return ((end.year - start.year) * 12) + (end.month - start.month) 

In [6]:
# Filter out missing issue and payment dates
df = df[df.issue_d.notnull() & df.last_pymnt_d.notnull() & df.term.notnull()]

# Convert dates to datetime
df.issue_d = pd.to_datetime(df.issue_d)
df.last_pymnt_d = pd.to_datetime(df.last_pymnt_d)
df.earliest_cr_line = pd.to_datetime(df.earliest_cr_line)

# Add maturity
df.term = df.term.str.replace(" months","").astype(int)
months = map(lambda x: pd.offsets.MonthBegin(x + 1,'M'), df.term)
df.maturity =  pd.Series(df.issue_d + np.array(months))

# Convert interest rates to float
df.int_rate = pd.Series(df.int_rate).str.replace('%', '').astype(float)
df.revol_util = pd.Series(df.revol_util).str.replace('%', '').astype(float)

# Convert employment length to number
df.emp_length.replace(to_replace='n/a', value=np.nan, inplace=True)
df.emp_length.fillna(value=0, inplace=True)
df.emp_length.replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df.emp_length = df.emp_length.astype(int)

# Convert earliest credit line field to months
df['credit_history'] = pd.Series(
    [mdiff(start,end) for start, end in zip(df.earliest_cr_line, df.issue_d)], index=df.index)


/Users/Branko/miniconda2/lib/python2.7/site-packages/pandas/core/ops.py:477: PerformanceWarning: Adding/subtracting array of DateOffsets to Series not vectorized
  "Series not vectorized", PerformanceWarning)


In [7]:
# Since we'll be looking at predicting loan performance, we'll limit the data set to only matured loans
as_of_date = max(df.last_pymnt_d)
df = df[df.maturity <= as_of_date]

# Add bad-loan to signify whether loan has taken charge-off
df["bad_loan"] = pd.Series(df.total_rec_prncp < df.funded_amnt)

df_fully_paid = df[df.bad_loan == False]
df_charged_off = df[df.bad_loan == True]

In [8]:
# Calculating Returns

# Add total return
df['total_return'] = pd.Series((df.total_rec_prncp + 
                                        df.total_rec_int + 
                                        df.total_rec_late_fee) / df.funded_amnt - 1, index=df.index)

# Add number of payments made
df['num_payments_made'] = pd.Series(
    [max(1, mdiff(start,end)) for start, end in zip(df.issue_d, df.last_pymnt_d)], index=df.index)

# Add average payment
df['average_payment'] = pd.Series(
        ((df.total_rec_prncp + 
          df.total_rec_int + 
          df.total_rec_late_fee) / df.num_payments_made ), 
    index=df.index
)

# Monthly returns -- assuming equal monthly payments from issue date to last payment date
returns = np.array([np.rate(n, pmt, pv, 0) for n, pmt, pv in 
          zip(df.num_payments_made, df.average_payment, -df.funded_amnt)])

# Annualize returns (simple return, not compounding)
# Note that annualized returns don't really make sense with loans taking a loss
# For instance, if the loan only returns 50% over 2 years, the total return would be -0.5 and the annualized
returns = ( 1 + returns) ** 12 - 1
df['annual_return'] = pd.Series(returns, index=df.index)

In [11]:
filename = "df_cleaned_up.csv"
df.to_csv(filename)
print "Cleaned up csv has been saved to %s" % filename

Cleaned up csv has been saved to df_cleaned_up.csv
